DFM and Scene-to-scene Runs
--

Initialize DFM runs with remote sensed chl field, run until next RS image.


In [1]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")

In [4]:
from stompy import utils
import six
import shutil
import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
import subprocess
import os
%matplotlib notebook

import logging
log = logging.getLogger()

import re
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import gridspec
import xarray as xr

import numpy as np
import pandas as pd

from stompy.memoize import memoize
from stompy.grid import unstructured_grid
from stompy import utils, filters
from stompy.plot import plot_wkb


In [5]:
#six.moves.reload_module(dwaq)

In [ ]:
# How often do we have restart files from the existing DFM run? no restart files.
# Or, once modern parallel DFM is ready, should I just run a fresh hydro?
# how long did that run originally take? 39h.

In [21]:
if 1:
    # This is not the same run as I've used for previous tracer runs
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_16layer"    
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_16layer")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_16layer.hyd"))
    
if 0: # no longer exists!
    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_with_temp"
    dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_with_temp")
    hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_with_temp.hyd"))

In [81]:
def hydro_name(hydro): return "wy2022_16layer"

def configure_dwaq(): # May be outdated, esp. for chicago.
    # configure DWAQ:
    DELFT_SRC="/opt/software/delft/delwaq/precompiled_binaries/DFM1.6.2.49199/lnx64"
    #DELFT_SRC="/home/alliek/software/Delft3D-FM/64634"
    DELFT_SHARE=os.path.join(DELFT_SRC,"share","delft3d")
    DELFT_LIB=os.path.join(DELFT_SRC,"lib")

    os.environ['DELFT_SRC']=DELFT_SRC
    os.environ['DELFT_SHARE']=DELFT_SHARE
    
    if 'LD_LIBRARY_PATH' in os.environ:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB+":"+os.environ['LD_LIBRARY_PATH']
    else:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB
    
configure_dwaq()

class CommonSetup(object):
    """
    Common code for various tracers runs
    """
    name='common' # should overload
    hydro=None
    base_path=None # must be set!

    force=True # whether to allow re-using an existing run
    
    start_time=np.datetime64("2022-08-10 00:00")
    stop_time=np.datetime64("2022-08-12 00:00") # seems that even temperature run is dicey on last step
    
    integration_option="15.60" # if set, copied to WaqModel
    time_step=3000
    map_time_step=3000 # otherwise it will default to time_step, which could be really short.    

    waq_kws={}
    def __init__(self,**kw):
        utils.set_keywords(self,kw)   
        if self.base_path is None:
            self.base_path=self.calc_base_path()
                
            yyyymmdd=utils.to_datetime(self.start_time).strftime('%Y%m%d')
            self.base_path+="_%s"%(yyyymmdd)   
            
            # And make it unique on successive runs
            for seq in range(50):
                test_path=self.base_path
                if seq>0:
                    test_path+=f"-v{seq:03}"
                if not os.path.exists(test_path):
                    self.base_path=test_path
                    break
            else:
                raise Exception("Too many runs with same name")
            
            log.info("base_path defaults to %s"%self.base_path)
    def calc_base_path(self):
        p='run_%s_%s'%(hydro_name(self.hydro),self.name)
        return p
        
    def release_conc_2d(self,X0,L):
        grid=self.hydro.grid()
        X=grid.cells_centroid()
        
        c=np.exp( -((X-X0)**2).sum(axis=-1)/L**2 )
        c=c/c.max() # make max value 1
        return c
    def release_conc_3d(self,*a,**kw):
        C_2d=self.release_conc_2d(*a,**kw)
        C_3d=self.hydro.extrude_element_to_segment(C_2d)
        return C_3d

    def setup_model(self):
        # Create a WaqModel, add some tracers
        self.wm=wm=dwaq.WaqModel(hydro=self.hydro,
                                 overwrite=True,
                                 base_path=self.base_path,
                                 mon_time_step=1000000, # daily
                                 map_time_step=self.map_time_step,
                                 time_step=self.time_step,
                                 **self.waq_kws)
        # add some option for balances.
        wm.integration_option="""%s ;
    LOWER-ORDER-AT-BOUND NODISP-AT-BOUND
    BALANCES-OLD-STYLE BALANCES-GPP-STYLE
    BAL_NOLUMPPROCESSES BAL_NOLUMPLOADS BAL_NOLUMPTRANSPORT
    BAL_NOSUPPRESSSPACe BAL_NOSUPPRESSTIME"""%self.integration_option
        #wm.start_time+= self.start_offset
        wm.start_time = self.start_time # may have to be smarter about starting on an output time step.
        # hydro reports the wrong stop time. manually set.
        if self.stop_time is not None:
            wm.stop_time=self.stop_time
        
        self.setup_tracers()
        
        wm.parameters['ACTIVE_VertDisp']=1
        wm.parameters['ScaleVDisp']=1.0 
        
    def run_waq_model(self):
        assert self.base_path is not None,"Must specify base_path"
        
        if not self.force:
            if os.path.exists(os.path.join(self.base_path,'dwaq_map.nc')):
                log.info("Run seems to exist -- will not run again")
                self.wm=dwaq.WaqModel.load(self.base_path,load_hydro=False)
                return

        self.setup_model()
        
        wm=self.wm
        wm.cmd_write_hydro()
        wm.cmd_write_inp()
        self.copy_notebook()        
        wm.cmd_delwaq1()
        wm.cmd_delwaq2()
        wm.cmd_write_nc()
    def copy_notebook(self):
        script_fn="run_dfm_vs_dwaq_swimming.ipynb"
        shutil.copyfile(script_fn,os.path.join(self.base_path,script_fn))
#     def setup_tracer_continuity(self):
#         # continuity tracer:
#         self.wm.substances['continuity']=dwaq.Substance(initial=1.0)
#         # This adds a concentration=1.0 boundary condition on all the boundaries.
#         all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]
#         self.wm.add_bc(all_bcs,'continuity',1.0)
    


In [82]:
# Instantaneous release, just see how the blob moves.
# Rather than using "anonymous" tracers as in the age tracer code, easier 
# to use substances that already have settling defined, but with no
# other processes. Use AlgNN tracers since there are lots of them.

class DwaqSwimAndPoint(CommonSetup):
    swim_speeds=[-15.0] # positive down.
    # close to the start in case we have to run DFM from cold start.
    # But enough into the run that tides are spun up, and hopefully any
    # initial baroclinic adjustments are done.
    start_time=np.datetime64("2022-05-10 00:00")
    stop_time=np.datetime64("2022-05-13 00:00") 
    name="dwaq_swim_point"
    
    point=[561470,4.16814e6]
    def setup_tracers(self):
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]

        for swim_i,speed in enumerate(self.swim_speeds):
            name=f'Alg{swim_i+1:02d}'
            conc=f'BLOOM' + name
            # initial condition of 1.0
            unity=1.0
            self.wm.substances[conc]=dwaq.Substance(initial=unity)
            self.wm.parameters['VSed' + name]=  speed             
            self.wm.add_process('SED' + name)        
            
            # This adds a concentration=1.0 boundary condition on all the boundaries.
            self.wm.add_bc(all_bcs,conc,unity)

        self.wm.parameters['TaucS']=0.0 # no deposition - covers all algae.
        
        # And a non-swimming, instantaneous point release. 
        
        # boundary condition will default to 0.0
        if 1: # is this causing the segfault? seems to. 
            grid=self.hydro.grid()
            C_2d=np.zeros(grid.Ncells(),np.float32) 
            C_2d[grid.select_cells_nearest(self.point)] = 10000.0
            C_3d=self.hydro.extrude_element_to_segment(C_2d)
            # HERE using that IC appears to trigger a seg fault. 
            self.wm.substances['dye1']=dwaq.Substance(initial=C_3d)

In [39]:
def configure_dwaq_new():
    DELFT_SRC="/opt/software/delft/dfm/2023.01"
    DELFT_SHARE=os.path.join(DELFT_SRC,"share","delft3d")
    DELFT_LIB=os.path.join(DELFT_SRC,"lib")

    os.environ['DELFT_SRC']=DELFT_SRC
    os.environ['DELFT_SHARE']=DELFT_SHARE

    # While mucking around with this just clobber whatever was in LD_LIBRARY_PATH
    os.environ['LD_LIBRARY_PATH']=DELFT_LIB

In [30]:
configure_dwaq_new()


INFO:WaqModel:NEFIS file didn't exist. Skipping ugrid_nef()


DFM Run with Newest DFM
==

In [26]:

def configure_dfm_t140737():
    DELFT_SRC="/opt/software/delft/dfm/t140737"
    DELFT_SHARE=os.path.join(DELFT_SRC,"share","delft3d")
    DELFT_LIB=os.path.join(DELFT_SRC,"lib")

    os.environ['DELFT_SRC']=DELFT_SRC
    os.environ['DELFT_SHARE']=DELFT_SHARE

    # While mucking around with this just clobber whatever was in LD_LIBRARY_PATH
    os.environ['LD_LIBRARY_PATH']=f"{DELFT_LIB}:/home/rusty/.conda/envs/dfm_t140737/lib"

configure_dfm_t140737()

# Try to leave as much the same as possible --
import stompy.model.delft.io as dio
import stompy.model.delft.dflow_model as dfm
six.moves.reload_module(dio)
six.moves.reload_module(dfm)

# And a DFM run to do something as similar as possible
# dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_16layer"

def copy_dfm_inputs(orig_path,new_path):
    """
    Copy DFM inputs
    This part is pretty slow (2 minutes?). Would be nice to avoid redoing it...
    """
    assert orig_path!=new_path
    shutil.copytree(orig_path, new_path,
                   ignore=shutil.ignore_patterns('*.dia','DFM_*','postprocessing'))


class RetargetedDfm(dfm.DFlowModel):
    """
    Various pieces of taking an existing run, adapting to more recent DFM
    version, and potentially adding some tracers and BCs.
    """
    orig_mdu=("/chicagovol1/hpcshared/open_bay/hydro/full_res/"
              "wy2022_bloom/runs/wy2022_bloom_16layer/wy2022_bloom_16layer.mdu")
    force_copy=False
    dfm_bin_dir="/opt/software/delft/dfm/t140737/bin"
    mpi_bin_dir="/opt/software/delft/dfm/t140737/bin"
    
    keep_partitions=True
    
    def __init__(self,**kw):
        self.saved_kw = kw
        super().__init__(**kw)
        
    def configure(self):
        super().configure()
        assert self.run_dir is not None
        assert self.run_dir != os.path.dirname(self.orig_mdu)
        if self.force_copy or not os.path.exists(self.run_dir):
            copy_dfm_inputs(self.orig_path,self.run_dir)
        
        self.load_from_mdu(os.path.join(self.run_dir, os.path.basename(self.orig_mdu)))
        utils.set_keywords(self,self.saved_kw)
        self.update_out_of_tree_paths()
        self.modernize_mdu()
        self.add_tracer_bcs()
        
    @property
    def orig_path(self):
        return os.path.dirname(self.orig_mdu)
    
    def update_out_of_tree_paths(self):
        # MDU references cross sections file that's outside the folder
        print("Updating out-of-tree relative paths")
        for entry in self.mdu.entries():
            idx,sec,key,value,comment = entry
            if value and "../" in value:
                real_path=os.path.abspath( os.path.join(self.orig_path,value) )
                sec=sec.replace('[','').replace(']','')
                model.mdu[sec,key]=real_path
                print(f"{value} => {real_path}")
 
        # And also in ext boundary file. 
        ext_fn=self.mdu.filepath(('external forcing','ExtForceFile'))
        orig_ext_fn=ext_fn+".orig"
        if not os.path.exists(orig_ext_fn):
            shutil.copyfile(ext_fn,orig_ext_fn)

        import re
        with open(orig_ext_fn,'rt') as fp_orig:
            with open(ext_fn,'wt') as fp_new:
                for line in fp_orig:
                    m=re.match(r'\s*filename\s*=\s*(\.\./[^#]+)(#.*)?',line,re.I)
                    if m:
                        print(line.strip())
                        real_path=os.path.abspath(os.path.join(self.orig_path,m.group(1)))
                        line=f"FILENAME={real_path} # updated to absolute\n"
                        print(" =>")
                        print(line.strip())
                        print()
                    fp_new.write(line)
            
    def modernize_mdu(self):
        # For recent DFM, have to drop a few mdu entries
        for sec_key in [
            ('numerics','transportmethod'),
            ('numerics','qhrelax'),
            ('numerics','transporttimestepping'),
            ('physics','effectspiral'),
            ('waves','knikuradse'),
            ('trachytopes','trtdt'),
            ('output','writebalancefile')
        ]:
            if sec_key in self.mdu:
                print("Drop old mdu setting",sec_key)
                del self.mdu[sec_key]

    def add_tracer_bcs(self,new_values=[]):
        """
        Source/sink BCs must have a column for each tracer. This
        is not sufficiently dynamic right now -- can move towards that...
        So if the new run will include two dwaq tracers, pass new_values=[0,1]
        (which would tag sources with 0 for the first and 1.0 for the second)
        """
        # And.... have to add a column to any source/sink files to get the
        # the additional tracers...
        bcs=self.load_bcs()
                
        orig_nvalues=3 # discharge, salinity, temperature

        # Fragile...
        for bc in bcs:
            if bc['QUANTITY'].upper()!='DISCHARGE_SALINITY_TEMPERATURE_SORSIN': continue
            pli_fn=os.path.join(self.run_dir,bc['FILENAME'])
            assert pli_fn.lower().endswith('.pli')
            fn=pli_fn[:-4] + ".tim"
            assert os.path.exists(fn)
            fn_orig=fn+".orig"
            if not os.path.exists(fn_orig):
                shutil.copyfile(fn,fn_orig)
            data_orig=np.loadtxt(fn_orig)
            columns=[data_orig]
            for new_val in new_values:
                columns.append( np.full(data_orig.shape[0],new_val))
            data=np.column_stack(columns)
            np.savetxt(fn,data,fmt="%.6g")
            
    def partition(self,partition_grid=None):
        if partition_grid is None:
            partition_grid=not self.keep_partitions
        super().partition(partition_grid=partition_grid)

In [27]:
model=RetargetedDfm(run_dir="dfm_spinup_repartition",
                    run_stop=np.datetime64("2022-08-01"),
                    keep_partitions=False)
model.configure()                 

Updating out-of-tree relative paths
../../sfb_dfm/inputs-static/deltabay.ldb => /chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/sfb_dfm/inputs-static/deltabay.ldb
../../sfb_dfm/inputs-static/SBlevees_tdk.pli => /chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/sfb_dfm/inputs-static/SBlevees_tdk.pli
../../sfb_dfm/inputs-static/SB-observationcrosssection.pli => /chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/sfb_dfm/inputs-static/SB-observationcrosssection.pli
FILENAME=../../sfb_dfm/inputs-static/friction12e.xyz
 =>
FILENAME=/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/sfb_dfm/inputs-static/friction12e.xyz
 # updated to absolute

Drop old mdu setting ('numerics', 'transportmethod')
Drop old mdu setting ('numerics', 'qhrelax')
Drop old mdu setting ('numerics', 'transporttimestepping')
Drop old mdu setting ('physics', 'effectspiral')
Drop old mdu setting ('trachytopes', 'trtdt')
Drop old mdu setting ('output', 'writebalancefile')
Encountered

In [28]:
if 0: # old IC stuff
    cc=model.grid.cells_centroid()
    conc=np.zeros(model.grid.Ncells())
    point=DwaqSwimAndPoint.point
    conc[model.grid.select_cells_nearest(point)] = 10000
    xyc=np.c_[cc,conc]
    blob_ic_fn="point-blob.xyz"
    np.savetxt(os.path.join(model.run_dir,blob_ic_fn),xyc,fmt="%11.1f")
    # And a spatially uniform one:
    xyc[:,2] = 1.0
    unity_ic_fn="uniform-blob.xyz"
    np.savetxt(os.path.join(model.run_dir,unity_ic_fn),xyc,fmt="%11.1f")


In [29]:
if 0: # run up to the start of the bloom without extra tracers.
    # And add conservative, non-swimming tracer. The spatial distribution will be
    # updated in a restart file later.
    fall_velocity=-15/86400.0

    with open(ext_fn,'at') as fp_ext:
        fp_ext.write(f"""

    QUANTITY=initialtracerblob
    FILENAME={blob_ic_fn}
    FILETYPE=7
    METHOD=5
    OPERAND=O

    """)
        fp_ext.write(f"""

    QUANTITY=initialtracerswim
    FILENAME={unity_ic_fn}
    FILETYPE=7
    METHOD=5
    OPERAND=O
    TRACERFALLVELOCITY={fall_velocity:.8f}
    """
    )

    # How to get a tracer field in there?
    model.mdu['external forcing','ExtForceFileNew']=""


In [30]:
# 12 day run is 9h or so.
# but for testing something very short -- just enough time to see if
# swimming is working.
# openmp but no mpi: 30 minutes for 6h.
#model.run_stop = model.run_start + np.timedelta64(6,'h')
model.mdu['output','mapinterval'] = 86400 
model.mdu['output','rstinterval'] = 10*86400
model.update_config()
model.write_config()
model.partition()

INFO:HydroModel:Could not find BC to get initial water level
INFO:DFlowModel:Writing MDU to dfm_spinup_repartition/wy2022_bloom_16layer.mdu
INFO:HydroModel:Running command: /opt/software/delft/dfm/t140737/bin/dflowfm --partition:ndomains=16:icgsolver=6 wy2022_bloom_16layer.mdu


In [ ]:
model.run_simulation()

INFO:HydroModel:Running command: /opt/software/delft/dfm/t140737/bin/mpiexec -n 16 /opt/software/delft/dfm/t140737/bin/dflowfm -t 1 --autostartstop wy2022_bloom_16layer.mdu


In [24]:
model.run_dir # should now be dfm_spinup_repartition

'dfm_spinup'

Plotting
==

In [31]:
ds=xr.open_dataset("run_wy2022_16layer_dfm_swim_and_point_20220810/DFM_OUTPUT_wy2022_bloom_16layer/wy2022_bloom_16layer_20220501_000000_map.nc")

In [32]:
ds

<xarray.Dataset>
Dimensions:                      (nmesh2d_EnclosurePoints: 7577,
                                  nmesh2d_EnclosureInstance: 1,
                                  nmesh2d_EnclosureParts: 16, nNetNode: 44184,
                                  nNetLink: 94194, nNetLinkPts: 2,
                                  nNetElem: 49996, nNetElemMaxNode: 6,
                                  nNetLinkContourPts: 4, nBndLink: 7561,
                                  nFlowElem: 49996, nFlowElemContourPts: 6,
                                  nFlowLink: 86840, nFlowLinkPts: 2, time: 97,
                                  laydim: 16, wdim: 17)
Coordinates:
    NetNode_x                    (nNetNode) float64 ...
    NetNode_y                    (nNetNode) float64 ...
    FlowElem_xcc                 (nFlowElem) float64 ...
    FlowElem_ycc                 (nFlowElem) float64 ...
    FlowLink_xu                  (nFlowLink) float64 ...
    FlowLink_yu                  (nFlowLink) float64 ...
  * time                         (time) datetime64[ns] 2022-05-01 ... 2022-05-13
    LayCoord_cc                  (laydim) float64 ...
    LayCoord_w                   (wdim) float64 ...
Dimensions without coordinates: nmesh2d_EnclosurePoints,
                                nmesh2d_EnclosureInstance,
                                nmesh2d_EnclosureParts, nNetNode, nNetLink,
                                nNetLinkPts, nNetElem, nNetElemMaxNode,
                                nNetLinkContourPts, nBndLink, nFlowElem,
                                nFlowElemContourPts, nFlowLink, nFlowLinkPts,
                                laydim, wdim
Data variables: (12/63)
    mesh2d_enc_x                 (nmesh2d_EnclosurePoints) float64 ...
    mesh2d_enc_y                 (nmesh2d_EnclosurePoints) float64 ...
    mesh2d_enc_node_count        (nmesh2d_EnclosureInstance) int32 ...
    mesh2d_enc_part_node_count   (nmesh2d_EnclosureParts) int32 ...
    mesh2d_enc_interior_ring     (nmesh2d_EnclosureParts) int32 ...
    mesh2d_enclosure_container   float32 ...
    ...                           ...
    vicwwu                       (time, nFlowLink, wdim) float64 ...
    tureps1                      (time, nFlowLink, wdim) float64 ...
    windx                        (time, nFlowElem) float64 ...
    windy                        (time, nFlowElem) float64 ...
    windxu                       (time, nFlowLink) float64 ...
    windyu                       (time, nFlowLink) float64 ...
Attributes:
    institution:    Deltares
    references:     http://www.deltares.nl
    source:         Deltares, D-Flow FM Version 1.2.167.141798, Oct 04 2022, ...
    history:        Created on 2023-03-03T15:27:44-0800, D-Flow FM
    date_created:   2023-03-03T15:27:44-0800
    date_modified:  2023-03-03T15:27:44-0800
    Conventions:    UGRID-0.9
    uuid:           a374a638-e7bd-4103-a5cd-a245f4fca343

In [33]:
g=unstructured_grid.UnstructuredGrid.read_ugrid(ds)

CHL RS Scenes
==